In [ ]:
# Importar bibliotecas necesarias para la red neuronal
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,LeakyReLU,Activation
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv(r'C:\Users\DAVID\Desktop\MECANICA_ROCAS_FREEFEM\MODELO_PREDICCION_SOSTENIMIENTO\DATA\datos_red_neuronal.csv')
df.drop(["Tipo_Roca","Historial_Fallos","Tipo_Roca_encoded","Tipo_Sostenimiento_Previo"], axis=1, inplace=True)


In [ ]:
df.head()

In [ ]:
columnas = df.columns.to_list()
columnas.pop(-1)
columnas

In [ ]:

# Seleccionar las características de entrada (X)
X = df[columnas]
X


In [ ]:

# La variable objetivo (y)
y = df['Tipo_Sostenimiento']
y

In [ ]:
# Normalización de los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Codificación de la variable objetivo para clasificación multiclase
y_encoded = to_categorical(y, num_classes=4)

# Dividir en conjunto de entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

# Definir el modelo de red neuronal
model = Sequential([
    Dense(128, input_dim=X_train.shape[1]),  # Capa de entrada
    LeakyReLU(alpha=0.1),  # Función de activación Leaky ReLU
    Dropout(0.5),
    Dense(64),
    LeakyReLU(alpha=0.1),
    Dropout(0.5),
    Dense(32),
    Dense(4, activation='softmax')  # Capa de salida con 4 neuronas (una para cada clase)
])

#! Cambiar el optimizador a RMSprop o Adam
optimizer = RMSprop(learning_rate=0.001)  # Puedes experimentar con el learning rate

# Compilar el modelo
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Definir EarlyStopping para evitar sobreajuste
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Precisión en el conjunto de prueba: {accuracy * 100:.2f}%")

# Graficar el desempeño del modelo durante el entrenamiento
plt.plot(history.history['accuracy'], label='Precisión en entrenamiento')
plt.plot(history.history['val_accuracy'], label='Precisión en validación')
plt.title('Precisión del Modelo durante el Entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.legend()
plt.show()

# Graficar la pérdida durante el entrenamiento
plt.plot(history.history['loss'], label='Pérdida en entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida en validación')
plt.title('Pérdida del Modelo durante el Entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.show()



In [ ]:
diccionario = {}
# Datos de entrada
for columna in columnas:
    value = float(input(f"Ingrese el valor para la columna '{columna}': "))
    diccionario[columna] = [value]


In [ ]:
# Hacer la predicción con nuevos datos
input_data = pd.DataFrame(diccionario)

# Normalizar los datos de entrada
input_data_scaled = scaler.transform(input_data)

# Hacer la predicción
prediccion = model.predict(input_data_scaled)
tipo_sostenimiento_predicho = np.argmax(prediccion, axis=1)

print(f"El tipo de sostenimiento predicho es: {tipo_sostenimiento_predicho[0]}")